# PHASE 3: ???
- TODO: fill in with Prof's description

# PLANNING

DATA PREP
- stratified split 

ML METHODS
- Binary classification
- Multiclass classification
- KNN?

EVALUATION
- Confusion Matrix: TP, FP, TN, FN, FP
  - Then F1 and stuff...????

SOURCE
- https://www.learndatasci.com/glossary/binary-classification/ 

# IMPORTS

In [195]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [196]:
#* TODO - it's only 3 seconds.. it should be fine to import all of it right?
#import sklearn
from sklearn.model_selection import train_test_split

# SETUP

### Import DF 
- From the CSV file I created in my Phase 2 (EDA) notebook

In [197]:
# Get current file path
current_path = os.getcwd()
parent_file_path = os.path.dirname(current_path)

# Construct desired file path
file_path = f'{parent_file_path}\\output\\oasis_cross-sectional_filtered.csv'
print('file_path =', file_path)

#* TODO Rename column names??
#* TODO -- set datatypes?
# Read CSV into Pandas, use the file path defined above, set row 0 as the header, and column 0 as the index
df = pd.read_csv(file_path, 
                 header = 0, 
                 index_col = 0)
df.head()

file_path = c:\Users\GlaDOS\Documents\GitHub\eugene_data606\output\oasis_cross-sectional_filtered.csv


,M/F,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF
0,F,74,2.0,3.0,29.0,0.0,1344,0.743,1.306
1,F,55,4.0,1.0,29.0,0.0,1147,0.810,1.531
2,F,73,4.0,3.0,27.0,0.5,1454,0.708,1.207
3,M,28,NaN,NaN,NaN,NaN,1588,0.803,1.105
4,M,18,NaN,NaN,NaN,NaN,1737,0.848,1.010


### Rename columns
- Elaborate on the full names according to the metadata descriptions provided

In [198]:
# For reference
df.columns

Index(['M/F', 'Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF'], dtype='object')

In [199]:
rename_dict = {'M/F'    : 'Gender', 
               'Educ'   : 'Education',
               'SES'    : 'Socioeconomic Status',
               'MMSE'   : 'Mini-Mental State Examination',
               'CDR'    : 'Clinical Dementia Rating',
               'eTIV'   : 'Estimated total intracranial volume (mm^3)',
               'nWBV'   : 'Normalized whole brain volume',
               'ASF'    : 'Atlas scaling factor'
                }

# Rename columns based on provided dictionary
df = df.rename(columns = rename_dict)

# display the updated dataframe
df

,Gender,Age,Education,Socioeconomic Status,Mini-Mental State Examination,Clinical Dementia Rating,Estimated total intracranial volume (mm^3),Normalized whole brain volume,Atlas scaling factor
0,F,74,2.0,3.0,29.0,0.0,1344,0.743,1.306
1,F,55,4.0,1.0,29.0,0.0,1147,0.810,1.531
2,F,73,4.0,3.0,27.0,0.5,1454,0.708,1.207
3,M,28,NaN,NaN,NaN,NaN,1588,0.803,1.105
4,M,18,NaN,NaN,NaN,NaN,1737,0.848,1.010
...,...,...,...,...,...,...,...,...,...
431,M,20,NaN,NaN,NaN,NaN,1469,0.847,1.195
432,M,22,NaN,NaN,NaN,NaN,1684,0.790,1.042
433,M,22,NaN,NaN,NaN,NaN,1580,0.856,1.111
434,F,20,NaN,NaN,NaN,NaN,1262,0.861,1.390


# PREPROCESSING

Q) Should I impute for my target column (CDR)? 

A) That doesn't sound like a good idea...

In [200]:
# CHECK how many unknowns in CDR
unknown_count = df['Clinical Dementia Rating'].isna().sum()
print(f'There are {unknown_count} unknowns in the CDR column')
print(f'Total length of CDR = {len(df)}')
print(f'Making that {int(unknown_count/len(df) * 100)}% of the CDR column missing')

There are 201 unknowns in the CDR column
Total length of CDR = 436
Making that 46% of the CDR column missing


### Drop missing rows in target column

In [201]:
# Drop rows with missing values in target column (CDR)
df.dropna(subset = ['Clinical Dementia Rating'], inplace=True)
df

,Gender,Age,Education,Socioeconomic Status,Mini-Mental State Examination,Clinical Dementia Rating,Estimated total intracranial volume (mm^3),Normalized whole brain volume,Atlas scaling factor
0,F,74,2.0,3.0,29.0,0.0,1344,0.743,1.306
1,F,55,4.0,1.0,29.0,0.0,1147,0.810,1.531
2,F,73,4.0,3.0,27.0,0.5,1454,0.708,1.207
8,M,74,5.0,2.0,30.0,0.0,1636,0.689,1.073
9,F,52,3.0,2.0,30.0,0.0,1321,0.827,1.329
...,...,...,...,...,...,...,...,...,...
411,F,70,1.0,4.0,29.0,0.5,1295,0.748,1.355
412,F,73,3.0,2.0,23.0,0.5,1536,0.730,1.142
413,F,61,2.0,4.0,28.0,0.0,1354,0.825,1.297
414,M,61,5.0,2.0,30.0,0.0,1637,0.780,1.072


### Split target column (CDR) into binary groups of non-demented or demented patients

In [202]:
# Set bin borders for integer to categorical conversion
bins = [0, 0.1, np.inf]

#* TODO - name this as the binary DF?
# Split the CDR into 2 groups: demented and non-demented
df['Clinical Dementia Rating'] = df['Clinical Dementia Rating'].apply(
    lambda x: 'non-demented' if x == 0.0 else (
        'demented' if x == 0.5 else(
            'demented' if x == 1 else(
                'demented' if x == 2 else 'unknown'))))

# CHECK - need to make sure that the lambda function split up CDR correctly
print("# of unique values in the CDR column =\n", df['Clinical Dementia Rating'].nunique(), '== 2?')
df.head(10)

# of unique values in the CDR column =
 2 == 2?


,Gender,Age,Education,Socioeconomic Status,Mini-Mental State Examination,Clinical Dementia Rating,Estimated total intracranial volume (mm^3),Normalized whole brain volume,Atlas scaling factor
0,F,74,2.0,3.0,29.0,non-demented,1344,0.743,1.306
1,F,55,4.0,1.0,29.0,non-demented,1147,0.810,1.531
2,F,73,4.0,3.0,27.0,demented,1454,0.708,1.207
8,M,74,5.0,2.0,30.0,non-demented,1636,0.689,1.073
9,F,52,3.0,2.0,30.0,non-demented,1321,0.827,1.329
11,F,81,5.0,2.0,30.0,non-demented,1664,0.679,1.055
13,M,76,2.0,NaN,28.0,demented,1738,0.719,1.010
14,M,82,2.0,4.0,27.0,demented,1477,0.739,1.188
16,M,39,3.0,4.0,28.0,non-demented,1636,0.813,1.073
17,F,89,5.0,1.0,30.0,non-demented,1536,0.715,1.142


# [1] Define explanatory and target variables
First, get the column names as a list

Then, pick the data columns and the target columns to use

In [203]:
# All columns besides the target column (CDR) will be used as the explanatory data columns
data_column_list = df.columns[df.columns != 'Clinical Dementia Rating'].tolist()
target_column_list = ['Clinical Dementia Rating']

# CHECK
print('type(data_column_list) =', type(data_column_list))
print('data_column_list =', data_column_list )
print('target_column_list =', target_column_list)

type(data_column_list) = <class 'list'>
data_column_list = ['Gender', 'Age', 'Education', 'Socioeconomic Status', 'Mini-Mental State Examination', 'Estimated total intracranial volume (mm^3)', 'Normalized whole brain volume', 'Atlas scaling factor']
target_column_list = ['Clinical Dementia Rating']


Split the DF up based on the predefined lists for data and target columns

In [204]:
X = df[data_column_list]
y = df[target_column_list]

# CHECK
print('X =\n', X)
print('y =\n', y)

X =
     Gender  Age  Education  Socioeconomic Status  \
0        F   74        2.0                   3.0   
1        F   55        4.0                   1.0   
2        F   73        4.0                   3.0   
8        M   74        5.0                   2.0   
9        F   52        3.0                   2.0   
..     ...  ...        ...                   ...   
411      F   70        1.0                   4.0   
412      F   73        3.0                   2.0   
413      F   61        2.0                   4.0   
414      M   61        5.0                   2.0   
415      F   62        3.0                   3.0   

     Mini-Mental State Examination  \
0                             29.0   
1                             29.0   
2                             27.0   
8                             30.0   
9                             30.0   
..                             ...   
411                           29.0   
412                           23.0   
413                         

# [2] Stratified train/test split

In [205]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X, 
                                        y, 
                                        test_size = 0.20,
                                        stratify = y,
                                        random_state = 0)

# CHECK
all_dfs_dict = {
    'X_test': X_test,
    'X_train': X_train,
    'y_test': y_test,
    'y_train': y_train
}

# Loop through the dictionary, print the name then each DF
for key, value in all_dfs_dict.items():
    print(f"{key}:\n{value}\n")

X_train =
     Gender  Age  Education  Socioeconomic Status  \
389      M   71        4.0                   1.0   
77       F   75        2.0                   3.0   
309      F   88        2.0                   3.0   
157      F   88        2.0                   3.0   
120      M   68        3.0                   3.0   
..     ...  ...        ...                   ...   
363      F   54        4.0                   3.0   
61       F   66        1.0                   4.0   
17       F   89        5.0                   1.0   
322      F   68        3.0                   2.0   
262      M   83        3.0                   2.0   

     Mini-Mental State Examination  \
389                           17.0   
77                            28.0   
309                           28.0   
157                           30.0   
120                           26.0   
..                             ...   
363                           29.0   
61                            28.0   
17                    

# [3] Normalize data

Q) When should I normalize?

A) When:
- distribution is not normal --> yes
- magnitudes are very different --> yes
- test and find out before/after

### CHECK
Q) How much of a difference would it be if I removed all rows with any nulls?

A) It will cut down the dataset around half (50%) according to my EDA notebook

### TODO
- maybe I should run both methods to check anyway?

### Impute missing data in rows
- IMPORTANT!!! --> run imputation AFTER train/test splitting

TODO:
- Impute missing data using median since most of these started as multi-class labels??? can't used mean for them anyway, and the distribution is very skewed according to my EDA notebook

In [209]:
# CHECK - before 
nulls_before = df.isnull().sum()

# Save a list of numerical (float/int) type columns
numerical_list = df.select_dtypes(include = 'number').columns.tolist()
print('numerical_list =', numerical_list)

# Impute median values in every numerical column
for column in numerical_list:
    median = df[column].median()
    df[column].fillna(median, inplace = True)
    
# CHECK - after
nulls_after = df.isnull().sum()

# Print the before and after sum of null values
print('nulls_before =\n', nulls_before)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('nulls_after =\n', nulls_after)


numerical_list = ['Age', 'Education', 'Socioeconomic Status', 'Mini-Mental State Examination', 'Estimated total intracranial volume (mm^3)', 'Normalized whole brain volume', 'Atlas scaling factor']
nulls_before =
 Gender                                         0
Age                                            0
Education                                      0
Socioeconomic Status                          19
Mini-Mental State Examination                  0
Clinical Dementia Rating                       0
Estimated total intracranial volume (mm^3)     0
Normalized whole brain volume                  0
Atlas scaling factor                           0
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
nulls_after =
 Gender                                        0
Age                                           0
Education                                     0
Socioeconomic Status                          0
Mini-Mental State Examination                 0
Clinical Dementia Rating   

# SAVE FOR LATER ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [208]:
# # Split the CDR into multiclass groups denoting the severity of the dementia
# df['CDR'] = df['CDR'].apply(
#     lambda x: 'non-demented' if x == 0.0 else (
#         'very mild dementia' if x == 0.5 else(
#             'mild dementia' if x == 1 else(
#                 'moderate dementia' if x == 2 else 'unknown'))))